# DSPy

## What?
Framework to algorithmically optimizing LM prompts and weights.

## Why?
Using LMs to build a complex system we generally have to: 
- Break the problem down into steps
- Prompt your LM well until each step works well in isolation
- Tweak the steps to work well together
- Generate synthetic examples to tune each step
- Use these examples to finetune smaller LMs to cut costs.

Currently, this is hard and messy: every time you change your pipeline, your LM, or your data, all prompts (or finetuning steps) may need to change.

## How?
DSPy breaks this process into following three abstractions:
- **Signatures:** Abstract the input and output behaviour
- **Modules:** Defines the flow of your program and sets up a pipeline. E.g of modules: `Predict`, `ChainOfThought`, `ProgramOfThought`, `MultiChainComparison` and `React`.
- **Optimizers:** Also known as `Teleprompters`, it takes the program, a training set and an evaluation metric and returns a new optimized program for the required use-case. Used to train smaller LMs (student) using larger LMs (teacher).

> Training Set can be small or have incomplete examples or without labels unless needed to be used in metric.
Metric can be `Exact Match (EM)` or `F1` or any custom defined metric

## DSPy Architecture

![dspy_arch](./assets/dspy_arch.png)

### Signature:
Use signature to tell, `what to do`, instead of `how to do`. Need not write huge prompts.
DSPy supports inline short strings as signatures, can always write custom classes for the same.

Some signatures available in DSPy:

| Task                        | Signature                      |
|-----------------------------|--------------------------------|
| Question-Answering         | "question -> answer"           |
| Summarization               | "document -> summary"          |
| Sentiment classification    | "sentence -> sentiment"        |
| RAG                         | "context, question -> answer"  |
| MCQs with Reasoning        | "question, choices -> reasoning, selection" |


### Module:
Takes the signature and converts it into a sophisticated prompt, based on a given technique and LLM used. Can be thought of as a model layer defined in Pytorch that learns from data (input/output).

### Optimizer:
DSPy optimizer can optimize 3 things
- LM weights
- Instructions (Prompt/Signature)
- Demonstrations of Input-Ouput Behaviour

Current available optimizers: [https://dspy-docs.vercel.app/docs/building-blocks/optimizers#what-dspy-optimizers-are-currently-available](https://dspy-docs.vercel.app/docs/building-blocks/optimizers#what-dspy-optimizers-are-currently-available)

#### Which optimizer should I use?
As a rule of thumb, if you don't know where to start, use `BootstrapFewShotWithRandomSearch`.

Here's the general guidance on getting started:
- If you have `very little data`, e.g. 10 examples of your task, use `BootstrapFewShot`.
- If you have `slightly more data`, e.g. 50 examples of your task, use `BootstrapFewShotWithRandomSearch`.
- If you have `more data than that`, e.g. 300 examples or more, use `MIPRO`.
- If you have been able to use one of these with a `large LM` (e.g., 7B parameters or above) and need a very efficient program, `compile` that down to a `small LM` with `BootstrapFinetune`.

## General Workflow
Whatever the task, the general workflow is:

- Collect a little bit of data.
- Define examples of the inputs and outputs of your program (e.g., questions and their answers). This could just be a handful of quick examples you wrote down. If large datasets exist, the more the merrier!
- Define the modules (i.e., sub-tasks) of your program and the way they should interact together to solve your task.
- Define some validation logic. What makes for a good run of your program? Maybe the answers need to have a certain length or stick to a particular format? Specify the logic that checks that.
- Compile! Ask DSPy to compile your program using your data. The compiler will use your data and validation logic to optimize your program (e.g., prompts and modules) so it's efficient and effective!Iterate.
- Repeat the process by improving your data, program, validation, or by using more advanced features of the DSPy compiler.

## Demo - RAG (Unoptimized)

We shall try RAG on truefoundry docs that are ingested in local docker based Qdrant deployment 

In [1]:
# First need the embedding
from embeddings.mixedbread import MixBreadEmbeddings
embedding_model = MixBreadEmbeddings(
    model_name="mixedbread-ai/mxbai-embed-large-v1"
)

In [2]:
# Set up Retriever
from vectordb.qdrant import CustomQdrantRetriever, QdrantClient

qdrant_client = QdrantClient(url="http://localhost:6333")
retriever = CustomQdrantRetriever(
    qdrant_collection_name="creditcard", 
    qdrant_client=qdrant_client, 
    embedding_model=embedding_model,
    k=5,
)


In [3]:
# also have other keys like metadata & score
retriever("What is creditcard", k=2)

  0%|          | 0/1 [00:00<?, ?it/s]

[{'long_text': '# [Regalia gold credit card](https://www.hdfcbank.com/personal/pay/cards/credit-cards/regalia-gold-credit-card)\n## Features\n#### Contactless Payment\nContactless Payment\nThe HDFC Bank Regalia Gold Credit Card is enabled for contactless payments, facilitating fast, convenient and secure payments at retail outlets.\n\\*To see if your Card is contactless, look for the contactless network symbol on your Card.\n(Please note that in India, payment through contactless mode is allowed for a maximum of ₹5000 in a single transaction without being asked to input your [Credit Card PIN](/personal/pay/cards/credit-cards/forgot-card-pin "/personal/pay/cards/credit-cards/forgot-card-pin"). However, if the amount is higher than or equal to ₹5000, the Cardholder has to enter the Credit Card PIN for security reasons.)'},
 {'long_text': '# [Infinia credit card](https://www.hdfcbank.com/personal/pay/cards/credit-cards/infinia-credit-card)\n## Features\n#### Insurance/Comprehensive protec

In [4]:
# Set up LLM
from dspy import OllamaLocal

llm = OllamaLocal(
    model="llama3:8b", 
    model_type="text", 
    max_tokens=1024, 
    top_p=1, 
    top_k=20, 
    temperature=0.1,
    base_url="http://localhost:11434", 
    frequency_penalty=0.9,
    presence_penalty=2
)

In [5]:
# Configure the settings
import dspy
dspy.configure(lm=llm, rm=retriever)

In [6]:
# Generate Signature for Input
class GenerateAnswer(dspy.Signature):
    """Answer the question in detail using the context."""

    context = dspy.InputField(desc="Contains relevant facts to answer the question")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="Detailed answer with respect to given question and context")

In [7]:
class RAG(dspy.Module):

    def __init__(self, k: int = 5):
        super().__init__()
        
        self.k = k
        self.retriever = dspy.Retrieve(k=self.k)
        self.generate_answer = dspy.ChainOfThought(signature=GenerateAnswer)

    def forward(self, question, k=None):
        passages = self.retriever(question, k).passages
        # print("Passages: ", passages)
        prediction = self.generate_answer(context=passages, question=question)
        return dspy.Prediction(context=passages, answer=prediction.answer)

In [8]:
# Ask any question you like to this simple RAG program.
my_question = "Explain different types of credit cards?"
uncompiled_rag = RAG(k=10)
# Get the prediction. This contains `pred.context` and `pred.answer`.
prediction = uncompiled_rag(my_question)

print(f"ANSWER: {prediction.answer}")
print("====================================\n\n")

for context in zip(prediction.context):
    print(f"Context: {context}")
    print("\n\n====================================")


  0%|          | 0/1 [00:00<?, ?it/s]

ANSWER: There are several types of credit cards offered by banks like HDFC Bank, each catering to different segments of customers with unique features and benefits.

Here's a breakdown:

* Infinia Credit Card (HDFC Bank): A premium card offering reward points redemption and cashback offers.
* Regalia Gold Credit Card (HDFC Bank): A high-end card providing smart EMI options for converting big spends into easy installments.
* Diners Club Black: A travel-focused credit card with revolving credit at a nominal interest rate, exclusive airport lounge access, and other benefits.

The HDFC Bank UPI RuPay Credit Card is specifically designed to provide:

1. Cashpoints:
	* 3% cashback on groceries, supermarkets & dining
	* 2% on utility spends (Maximum of ₹500 in a calendar month)
	* 1% on other transactions (excluding Rent, Wallet loads, EMI, Fuel, Insurance Payments & Government categories) (Maximum of ₹500 in a calendar month)

These cards cater to different customer segments:

* Infinia: For

In [9]:
# Track llm history
llm.inspect_history(n=1)





Answer the question in detail using the context.

---

Follow the following format.

Context: Contains relevant facts to answer the question

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: Detailed answer with respect to given question and context

---

Context:
[1] «# [Freedom card new](https://www.hdfcbank.com/personal/pay/cards/credit-cards/freedom-card-new)
## Features
#### Additional Features
Additional Features
**Zero lost Card liability :** In the unfortunate event of losing your HDFC Bank Freedom Credit Card, report it immediately to our 24-hour call centre. On reporting the loss immediately, you have zero liability on any fraudulent transactions made on your Credit Card.
**Interest Free Credit Period :** Avail up to 50 days of interest free period on your HDFC Bank Freedom Credit Card from the date of purchase (subject to the submission of the charge by the Merchant).
**Revolving Credit :** Enjoy Revolving Cred

In [10]:
# Create Train and Test examples - Can also use HF Datasets
train = [
    {
        "question" : "What factors influence the dynamic adjustment of the number of replicas in autoscaling, and how is the autoscaling strategy determined based on the Queue Backlog?",
        "answer" : "The dynamic adjustment of the number of replicas in autoscaling is influenced by the fluctuation in traffic or resource usage. Specifically, the autoscaling strategy is determined based on the Queue Backlog. When traffic or resource usage varies, the autoscaling strategy evaluates the Queue Backlog to decide the appropriate number of replicas between the defined minimum and maximum replica counts."
    }, 
    {
        "question" : "What role do autoscaling metrics play in dynamically adjusting resource allocation for an async service?",
        "answer" : "Autoscaling metrics play a crucial role in dynamically adjusting resource allocation for an async service by monitoring and responding to changing demands while maintaining optimal performance. These metrics, such as AWS SQS Average Backlog, provide insights into the queue length and help the autoscaler determine the appropriate resource allocation to handle incoming requests efficiently."
    },
    {
        "question" : "What is the purpose of creating teams within the Truefoundry platform, and how does it streamline resource management?",
        "answer" : "Creating teams within the Truefoundry platform serves to streamline resource management by simplifying access control and allocation processes. Teams allow users to group individuals with similar responsibilities or access requirements, reducing the need to individually assign permissions for each resource. Additionally, teams facilitate efficient collaboration by providing a structured framework for managing access permissions across multiple resources."
    },
    {
        "question" : "How does the Truefoundry platform support role-based access control (RBAC) for managing user permissions?",
        "answer" : "The Truefoundry platform supports role-based access control (RBAC) by assigning specific roles to users based on their responsibilities and access requirements. Each role defines a set of permissions that determine the actions users can perform on resources within the platform. By assigning roles to users, administrators can enforce security policies, restrict unauthorized access, and ensure compliance with data protection regulations."
    },
    {
        "question" : "What are the key benefits of using the Truefoundry platform for managing cloud resources?",
        "answer" : "The Truefoundry platform offers several key benefits for managing cloud resources, including centralized resource management, automated provisioning, and enhanced security features. By providing a unified interface to monitor and control cloud resources, Truefoundry simplifies resource allocation, reduces operational overhead, and improves scalability. Additionally, automated provisioning capabilities streamline resource deployment processes, while security features such as RBAC and encryption enhance data protection and compliance."
    },
    {
        "question" : "What are the steps involved in creating your account on TrueFoundry?",
        "answer" : "Navigate to the `create your account` page on the TrueFoundry website. Fill out the registration form with your company name, work email, username, and password. Click the `Create Account` button to submit the form."
    },
    {
        "question" : "What cloud providers are supported for creating Kubernetes clusters on TrueFoundry, and what are the recommended options for accessing all platform features?",
        "answer" : "TrueFoundry supports AWS EKS, GCP GKE, and Azure AKS for creating Kubernetes clusters. For accessing all platform features, it is recommended to use one of these major cloud providers. Note that kind and minikube, while supported for local clusters, may not support all platform features."
    },
    {
        "question" : "What are TrueFoundry jobs, and in what scenarios are they particularly well-suited?",
        "answer" : "TrueFoundry jobs are task-oriented workloads designed to run for a specific duration to complete a task and then terminate, releasing the resources. They are well-suited for scenarios such as model training on large datasets, routine maintenance tasks like data backups and report generation, and large-scale batch inference tasks."
    },
    {
        "question" : "What is an MLRepo in Truefoundry, and how does it differ from Git repositories?",
        "answer" : "An MLRepo in Truefoundry serves the purpose of versioning ML models, artifacts, and metadata, similar to how Git repositories version code. However, MLRepos are specifically tailored for ML assets, and access to them can be granted to workspaces, enabling secure and controlled access to ML assets across teams and applications."
    }
]


test = [
    {
        "question" : "How can you view the details of a job run in TrueFoundry?",
        "answer" : "You can view the details of a job run in TrueFoundry by accessing the Job Run section. This section provides information about the status and progress of the job run"
    },
    {
        "question" : "What are key design principles of truefoundry?",
        "answer" : "The key design principles of TrueFoundry are: Cloud Native: TrueFoundry operates on Kubernetes, allowing it to function on various cloud providers or on-premises environments. ML Inherits the same SRE principles as the rest of the infrastructure: TrueFoundry seamlessly integrates with your existing software stack, providing ML teams with the same SRE (Site Reliability Engineering), security, and cost optimization features. No Vendor Lockin: TrueFoundry is designed to avoid vendor lock-in. It ensures easy migration by providing accessible APIs and exposing all Kubernetes manifests generated, enabling smooth transition if needed."
    },
    {
        "question" : "What architecture does TrueFoundry follow, and what benefits does it offer?",
        "answer" : "TrueFoundry follows a split-plane architecture, enabling both on-premises deployment and ensuring that service reliability does not rely solely on TrueFoundry. This architecture enhances reliability and flexibility while allowing for customization based on specific organizational needs."
    },
    {
        "question" : "How is the organization of workspaces typically structured within a cluster?",
        "answer" : "Workspaces within a cluster can be organized based on teams, applications, and environments. For example, different teams may manage various applications, each with its own set of environments such as development, staging, and production."
    },
    {
        "question" : "How can a user create a workspace in Truefoundry?",
        "answer" : "To create a workspace in Truefoundry, users can navigate to the Workspace tab in the platform and click on the `Create Workspace` button. Once created, users can obtain the Fully Qualified Name (FQN) of the workspace from the FQN button."
    },
    {
        "question" : "What is the process for creating an ML Repo in Truefoundry?",
        "answer" : "To create an ML Repo in Truefoundry, users need to have at least one Storage Integration configured. They can then access the list of storage integrations from the dropdown menu and select one to associate with the ML Repo. After selecting a storage integration, users can create an ML Repo from the ML Repo's tab in the platform."
    },
]

In [11]:
from dspy import Example

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [Example(**data).with_inputs('question') for data in train]
testset = [Example(**data).with_inputs('question') for data in test]

In [12]:
trainset[0]

Example({'question': 'What factors influence the dynamic adjustment of the number of replicas in autoscaling, and how is the autoscaling strategy determined based on the Queue Backlog?', 'answer': 'The dynamic adjustment of the number of replicas in autoscaling is influenced by the fluctuation in traffic or resource usage. Specifically, the autoscaling strategy is determined based on the Queue Backlog. When traffic or resource usage varies, the autoscaling strategy evaluates the Queue Backlog to decide the appropriate number of replicas between the defined minimum and maximum replica counts.'}) (input_keys={'question'})

In [13]:
len(trainset), len(testset)

(9, 6)

In [14]:
from dspy.teleprompt import BootstrapFewShot

# Validation logic: check that the predicted answer is correct.
# Also check that the retrieved context does actually contain that answer.
def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

In [15]:
# Set up a basic optimizer, which will compile our RAG program.
optimizer = BootstrapFewShot(metric=validate_context_and_answer)

In [16]:
uncompiled_rag.named_predictors()

[('generate_answer',
  ChainOfThought(GenerateAnswer(context, question -> answer
      instructions='Answer the question in detail using the context.'
      context = Field(annotation=str required=True json_schema_extra={'desc': 'Contains relevant facts to answer the question', '__dspy_field_type': 'input', 'prefix': 'Context:'})
      question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
      answer = Field(annotation=str required=True json_schema_extra={'desc': 'Detailed answer with respect to given question and context', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
  )))]

In [17]:
# Compile!
compiled_rag = optimizer.compile(RAG(), trainset=trainset)

  0%|                                                                                                                                                  | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

 11%|███████████████▎                                                                                                                          | 1/9 [00:09<01:17,  9.71s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

 22%|██████████████████████████████▋                                                                                                           | 2/9 [00:20<01:11, 10.27s/it]

...

  0%|          | 0/1 [00:00<?, ?it/s]

 89%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋               | 8/9 [01:11<00:10, 10.28s/it]

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:31<00:00, 10.13s/it]

Bootstrapped 0 full traces after 9 examples in round 0.


In [18]:
from dspy.evaluate.evaluate import Evaluate
# Set up the `evaluate_on_dataset` function. We'll use this many times below.
evaluate_on_dataset = Evaluate(devset=trainset, num_threads=4, display_progress=True, display_table=5)
evaluate_on_dataset(compiled_rag, metric=validate_context_and_answer)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|                                                                                                                                                  | 0/9 [00:00<?, ?it/s]



Average Metric: 0 / 1  (0.0):   0%|                                                                                                                    | 0/9 [00:46<?, ?it/s]



Average Metric: 0 / 1  (0.0):  11%|████████████                                                                                                | 1/9 [00:46<06:08, 46.05s/it]

...





Average Metric: 0 / 6  (0.0):  56%|████████████████████████████████████████████████████████████                                                | 5/9 [02:08<01:35, 23.90s/it]



Average Metric: 0 / 6  (0.0):  67%|████████████████████████████████████████████████████████████████████████                                    | 6/9 [02:08<00:52, 17.56s/it]



Average Metric: 0 / 7  (0.0):  67%|████████████████████████████████████████████████████████████████████████                                    | 6/9 [02:18<00:52, 17.56s/it]



Average Metric: 0 / 7  (0.0):  78%|████████████████████████████████████████████████████████████████████████████████████                        | 7/9 [02:18<00:30, 15.11s/it]



Average Metric: 0 / 8  (0.0):  78%|████████████████████████████████████████████████████████████████████████████████████                        | 7/9 [02:22<00:30, 15.11s/it]



Average Metric: 0 / 8  (0.0):  89%|████████████████████████████████████████████████████████████████████████████

Average Metric: 0 / 9  (0.0%)



/Users/prathamesh/TrueFoundry/learn-dspy/venv/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' 'False' 'False' 'False' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,example_answer,context,pred_answer,validate_context_and_answer
0,"What factors influence the dynamic adjustment of the number of replicas in autoscaling, and how is the autoscaling strategy determined based on the Queue Backlog?","The dynamic adjustment of the number of replicas in autoscaling is influenced by the fluctuation in traffic or resource usage. Specifically, the autoscaling strategy is...",['# [Indianoil hdfc bank credit card](https://www.hdfcbank.com/personal/pay/cards/credit-cards/indianoil-hdfc-bank-credit-card)\n## Features\n#### Reward Point/CashBack Redemption & Validity\n* Redeem Fuel Points through NetBanking towards catalogue products (where 1 FP = upto...,"I'm happy to help! The dynamic adjustment of the number of replicas in autoscaling is influenced by several factors, including: 1. **Queue Length**: The number...",False
1,What role do autoscaling metrics play in dynamically adjusting resource allocation for an async service?,Autoscaling metrics play a crucial role in dynamically adjusting resource allocation for an async service by monitoring and responding to changing demands while maintaining optimal...,['# [Indianoil hdfc bank credit card](https://www.hdfcbank.com/personal/pay/cards/credit-cards/indianoil-hdfc-bank-credit-card)\n## Features\n#### Reward Point/CashBack Redemption & Validity\n* Redeem Fuel Points through NetBanking towards catalogue products (where 1 FP = upto...,Autoscaling Metrics Play Step AutosScaling Monitor LatThroughRequestLatCPUUsageMemoryUtilReasoningLetStepAsyncServiceInstanceautoscale autoscale async service instance autoscaled monitor performance request latency through memory util reasonings let step Async Service Instance...,False
2,"What is the purpose of creating teams within the Truefoundry platform, and how does it streamline resource management?",Creating teams within the Truefoundry platform serves to streamline resource management by simplifying access control and allocation processes. Teams allow users to group individuals with...,"['# [Infinia credit card](https://www.hdfcbank.com/personal/pay/cards/credit-cards/infinia-credit-card)\n## Features\n#### Reward Point/CashBack Redemption & Validity\n\\*Please\xa0[click here](/Personal/Pay/Cards/Credit Card/Credit Card Landing Page/Credit Cards/Super Premium/Infinia/RP-Program-TnC.pdf ""/Personal/Pay/Cards/Credit Card/Credit Card Landing Page/Credit Cards/Super Premium/Infinia/RP-Program-TnC.pdf"")\xa0to know...","I apologize for the confusion earlier. It seems like you didn't provide any information about Truefoundry or teams within it. However, if we assume a...",False
3,How does the Truefoundry platform support role-based access control (RBAC) for managing user permissions?,The Truefoundry platform supports role-based access control (RBAC) by assigning specific roles to users based on their responsibilities and access requirements. Each role defines a...,['# [Infinia credit card](https://www.hdfcbank.com/personal/pay/cards/credit-cards/infinia-credit-card)\n## Features\n#### Good Food Trail Program\nGood Food Trail Program\nEnjoy amazing dining benefits with Good Food Trail program\nGet\xa0additional 15% off on all your...,"I'm happy to help! However, as mentioned earlier, there is no mention of ""Truefoundry"" or any related information. It seems like the text provided might...",False
4,What are the key benefits of using the Truefoundry platform for managing cloud resources?,"The Truefoundry platform offers several key benefits for managing cloud resources, including centralized resource management, automated provisioning, and enhanced security features. By providing a unified...","['# [Marriott bonvoy credit card](https://www.hdfcbank.com/personal/pay/cards/credit-cards/marriott-bonvoy-credit-card)\n## Features\n#### Welcome Benefit\nWelcome Benefit\n1 Free Night Award on Joining fee realization (up to a value of 15,000 Marriott Bonvoy Points)\nComplimentary...","I think there may be some confusion here

0.0

In [19]:
# Ask any question you like to this simple RAG program.
my_question = "Explain different types of storage classes?"
# Get the prediction. This contains `pred.context` and `pred.answer`.
prediction = compiled_rag(my_question, k=10)

print(f"Answer: {prediction.answer}")
print("====================================\n\n")

for context in zip(prediction.context):
    print(f"Context: {context}")
    print("\n\n====================================")


  0%|          | 0/1 [00:00<?, ?it/s]

Answer: Here's an explanation of the main types of storage classes:

**Stack**: A stack follows the LIFO principle, where data is added to and removed from one end only. This means that when you add a new element (like a plate), it goes on top, and when you remove an element, it comes off last.

Example: Think about stacking plates - each time you put down another plate, it gets stacked on the previous ones. When serving dinner, we take them out in reverse order!

**Queue**: A queue follows the FIFO principle (First In First Out), where data is added to one end and removed from the other. This means that when a new element arrives, it goes at the front of the line.

Example: Picture waiting for your turn at a bank - you join behind others who arrived before you!

**Heap**: A heap stores elements in order based on their priority or value (like an emergency response system prioritizing calls by severity). This allows efficient retrieval and manipulation of data according to its importanc